# ⛓️ PyKwant Tutorial: Yield Curve Bootstrapping

This notebook introduces `pykwant.bootstrapping`, the module responsible for constructing Yield Curves from market quotes.

In the real world, we don't observe the Yield Curve directly. We observe prices of instruments like:
1. **Deposits** (Short term, simple interest).
2. **Futures/FRAs** (Medium term).
3. **Swaps** (Long term, par coupon rates).

Bootstrapping is the process of solving for the sequence of Zero Rates that perfectly reprices these input instruments.

## 1. Setup and Market Data

We define our valuation date and the market conventions.

In [6]:
from datetime import date

from pykwant import bootstrapping, dates, instruments, rates

# Valuation Date
today = date(2025, 1, 1)
cal = dates.Calendar(holidays=frozenset(), weekends=(6, 7))

## 2. Defining Calibration Instruments

We use `DepositRate` for the short end (Cash) and `SwapRate` for the long end.

**Market Quotes**:
* 6M Deposit: 3.0%
* 1Y Swap: 3.2%
* 2Y Swap: 3.5%
* 5Y Swap: 4.0%
* 10Y Swap: 4.5%

In [7]:
helpers = []

# --- 1. Short End (Deposit) ---
# 6 Months maturity
dep_maturity = date(2025, 7, 1)
helpers.append(bootstrapping.DepositRate(rate=0.030, maturity_date=dep_maturity))


# --- 2. Long End (Swaps) ---
# Helper to add swaps easily
def add_swap(years, rate):
    mat = date(today.year + years, today.month, today.day)
    helpers.append(
        bootstrapping.SwapRate(rate=rate, maturity_date=mat, frequency_months=12, calendar=cal)
    )


add_swap(1, 0.032)
add_swap(2, 0.035)
add_swap(5, 0.040)
add_swap(10, 0.045)

print(f"Defined {len(helpers)} calibration instruments.")

Defined 5 calibration instruments.


## 3. Running the Bootstrap

The `bootstrap_curve` function performs the iterative solving. It returns a standard `YieldCurveFn` compatible with the rest of the library.

In [8]:
curve = bootstrapping.bootstrap_curve(today, helpers)

print("Curve constructed successfully.")

Curve constructed successfully.


## 4. Analyzing the Term Structure

Let's extract the Zero Rates for each pillar to visualize the Term Structure of Interest Rates.

In [9]:
print(f"{'Maturity':<12} | {'Input Quote':<12} | {'Zero Rate':<12} | {'Discount Factor':<15}")
print("-" * 60)

for h in helpers:
    # Extract Zero Rate from the bootstrapped curve
    z_rate = rates.zero_rates(curve, today, h.maturity_date)
    df = curve(h.maturity_date)

    print(f"{str(h.maturity_date):<12} | {h.rate:>12.2%} | {z_rate:>12.4%} | {df:>14.6f}")

Maturity     | Input Quote  | Zero Rate    | Discount Factor
------------------------------------------------------------
2025-07-01   |        3.00% |      3.0190% |       0.985141
2026-01-01   |        3.20% |      3.1499% |       0.968992
2027-01-01   |        3.50% |      3.4452% |       0.933416
2030-01-01   |        4.00% |      3.9421% |       0.821014
2035-01-01   |        4.50% |      4.4719% |       0.639266


**Note**: For Swaps, the Zero Rate is slightly different from the Par Swap Rate (Input Quote) because the Swap Rate is a "weighted average" of the zero rates along the coupon schedule, whereas the Zero Rate is the spot rate for that specific maturity.

## 5. Verification: Repricing to Par

A correctly bootstrapped curve must reprice the input instruments exactly to Par (or 1.0).
Let's verify this for the **5-Year Swap**.

In [10]:
# Reconstruct the 5Y Swap as a Fixed Rate Bond
swap_5y_helper = helpers[3]  # Index 3 is the 5Y Swap (4.0%)

swap_instrument = instruments.FixedRateBond(
    face_value=instruments.Money(100.0),
    coupon_rate=swap_5y_helper.rate,
    start_date=today,
    maturity_date=swap_5y_helper.maturity_date,
    frequency_months=12,
    day_count=dates.thirty_360,
    calendar=cal,
)

# Price using the bootstrapped curve
npv = instruments.price_instrument(swap_instrument, curve, today)

print(f"5Y Swap Rate Input: {swap_5y_helper.rate:.2%}")
print("Target Price:       100.0000")
print(f"Calculated Price:   {npv:.4f}")

assert abs(npv - 100.0) < 1e-4
print("✅ Calibration Verified.")

5Y Swap Rate Input: 4.00%
Target Price:       100.0000
Calculated Price:   100.0000
✅ Calibration Verified.
